In [1]:
#!usr/bin/env python3

In [1]:
!pip install beautifulsoup4
!pip install requests
!pip install lxml

In [9]:
import bs4
import requests
import lxml
import re
import os

In [8]:
!wget -r -p -E -k -np -nd https://docs.aelf.com/ -P ./aelf_docs

--2024-07-29 20:45:17--  https://docs.aelf.com/
Resolving docs.aelf.com (docs.aelf.com)... 104.22.20.159, 172.67.29.22, 104.22.21.159, ...
Connecting to docs.aelf.com (docs.aelf.com)|104.22.20.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./aelf_docs/index.html’

index.html              [ <=>                ]  19.92K  --.-KB/s    in 0.04s   

2024-07-29 20:45:18 (509 KB/s) - ‘./aelf_docs/index.html’ saved [20397]

Loading robots.txt; please ignore errors.
--2024-07-29 20:45:18--  https://docs.aelf.com/robots.txt
Reusing existing connection to docs.aelf.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./aelf_docs/robots.txt.html’

robots.txt.html         [ <=>                ]  19.92K  --.-KB/s    in 0s      

2024-07-29 20:45:18 (100 MB/s) - ‘./aelf_docs/robots.txt.html’ saved [20397]

--2024-07-29 20:45:18--  https://docs.aelf.com/img/favicon.ico
Reusing existing conn

In [23]:
html_files = [filen for filen in os.listdir('./aelf_docs') if filen.endswith('.html')]
img_files = [filen for filen in os.listdir('./aelf_docs') if filen.endswith('.png') or filen.endswith('.svg')]
len(html_files), len(img_files)

(171, 64)

In [57]:
# prompt: set current working directory to 'aelf_docs'

os.chdir('aelf_docs')
os.getcwd()

'/content/aelf_docs'

In [58]:
os.listdir()[:4]

['index.html.1.102.html',
 'KaTeX_Size2-Regular.woff.html',
 'KaTeX_Main-Regular.ttf.html',
 'KaTeX_Fraktur-Bold.woff.html']

In [3]:
def clean_text(text):

    # Remove control characters
    text = text.encode("ascii", "ignore").decode("ascii")

    # Remove emojis using regex (optional)
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & Pictograms
        "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
        "\U00002500-\U00002BEF"  # Chinese, Japanese, Korean symbols & punctuation
        "\U00002702-\U000027B0"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u200b"
        "\u200c"
        "\u200e"
        "\u2069"
        "\u2066"
        "\u2067"
        "\u2068"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"  # Dingbats
        "\u3030"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    return text

In [59]:
def extract_content(html_content):
    html_content = open(html_content, 'r').read()
    soup = bs4.BeautifulSoup(html_content, 'lxml')

    out_list = []
    for element in soup.find_all(['p', 'h1', 'h2', 'h3', 'pre', 'code']):
        out_list.extend(clean_text(element.text) + "\n")

    soup_txt = ''.join(word for word in out_list).replace('*', '')
    return soup_txt

In [62]:
page_summary_prompt = f"""You are an assistant tailored for summarizing text for retrieval.
  These summaries would contain any useful information for a developer that wants to solve an issue.
  These summaries should contain full code snippets if any.
  Note that the summaries will be turned into vector embeddings and used to retrieve the raw text.
  Give a concise summary of the text that is well optimized for retrieval. Here is the text."""

In [51]:
!pip install -q -U google-generativeai chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 85.6 MB/s eta 0:00:00


In [60]:
import google.generativeai as genai


from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('models/gemini-1.5-flash')
# filer = genai.upload_file('out.txt')
# response = model.generate_content([page_summary_prompt, filer])

In [63]:
text_summaries = []

for f in html_files:
    response = model.generate_content([page_summary_prompt, extract_content(f)])

    text_summaries.append(response.text)

text_summaries

['The Treasury contract is used to distribute bonuses to voters and candidates during the election process.  The contract implements AElf standards ACS1 and ACS10. The contract has methods for donating, setting weights for the dividend pool and miner rewards, setting profits receivers, getting the welfare reward amount, recording miner replacements, replacing candidate profits receivers, and releasing dividends.  The contract also defines types for dividend pool weights, miner reward weights, donation inputs, welfare reward amounts, miner re-election information, scheme proportions, profits receivers, and dividends.\n```C#\npublic class TreasuryContract : TreasuryContractContainer.TreasuryContractBase\n{\n    public override async Task<Empty> DonateAll(DonateAllInput input)\n    {\n        // Logic for donating all tokens.\n    }\n\n    public override async Task<Empty> DividendPoolWeightSetting(DividendPoolWeightSetting input)\n    {\n        // Logic for setting dividend pool weights

In [65]:
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

In [67]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/text-embedding-004'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input,
                                task_type="retrieval_document",
                                title=title)["embedding"]

In [70]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.get_or_create_collection(name=name, embedding_function=GeminiEmbeddingFunction())

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

In [71]:
ldb = create_chroma_db(text_summaries, "aelf")

In [72]:
import pandas as pd
data = {
    'embeddings': ldb.peek()['embeddings'],
    'documents': ldb.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.09232533723115921, 0.037840984761714935, -0...",The Treasury contract is used to distribute bo...
1,"[0.02989785745739937, -0.02082676626741886, -0...","aelf is a high-performance, cloud-native, laye..."
2,"[0.02967061661183834, -0.01643938384950161, -0...","aelf is a high-performance, cloud-native block..."
3,"[0.038800764828920364, -0.03525588661432266, -...",This document lists available SDKs for the Ael...
4,"[0.02515622414648533, -0.002571708522737026, -...",The documentation describes a blockchain explo...
5,"[0.03891860321164131, -0.012901981361210346, -...","aelf is a high-performance, cloud-native, laye..."
6,"[0.038332197815179825, -0.016487376764416695, ...",## aelf-sdk.php - aelf PHP API\n\nThis documen...
7,"[0.056426890194416046, 0.00847102515399456, -0...",## Simulating Block Producer Elections in aelf...
8,"[0.031681280583143234, -0.02149266190826893, -...","## aelf: A High-Performance, Cloud-Native Bloc..."
9,"[0.05769975855946541, 0.007813281379640102, -0...",## aelf-web-login: React wallet integration li...


In [73]:
def get_relevant_files(query, db):
  results = db.query(query_texts=[query], n_results=3)
  return results["ids"][0]

In [74]:
filex = get_relevant_files("""Extra functionalities include:

    Serializing and deserializing requests.
    Implementing request/response mechanisms.
    Authentication.
""", ldb)
print(filex)

['144', '11', '138']


In [77]:
def query_rag(query, db):
  files = get_relevant_files(query, db)
  prompt = [text_summaries[int(f)] for f in files]
  prompt.append("Generate a response to the query using the provided files. Here is the query.")
  prompt.append(query)
  return model.generate_content(prompt).text, [html_files[int(f)] for f in files]

In [82]:
semi_result = query_rag("how to build a contract in aelf", ldb)

last_prompt = """You are a summarising agent. Summarise the following text to briefly explain the key idea outlined in the text without directly referring to the text.
                 Make your output like as if you were writing a blog post giving step-by-step guides to a developer.
                 Here is the text:"""

last_response = model.generate_content([last_prompt, semi_result[0].replace('#', '')])
print(last_response.text)

##  Building Smart Contracts on aelf: A Developer's Guide

Ready to dive into the world of smart contracts on the aelf blockchain? This guide will walk you through the process, from a simple "Hello" contract to more complex projects.

**Step 1: Set Up Your Workspace**

- Head over to the aelf Playground, a web-based sandbox designed for easy smart contract development.
- No need for local installations - just pick a template (like "Hello") and create your project!

**Step 2: Write Your Contract**

- The Playground opens a code editor with a pre-populated template.
- Customize your contract logic within this editor. For example, change the message returned by the "Hello" function.

**Step 3: Build Your Contract**

- Simply click the "Build" button in the Playground interface.
- Wait for the build process to complete - the Playground will show you the status.

**Step 4: Deploy Your Contract**

- aelf offers two deployment methods:
    - **With BP Approval:** Requires approval from Block 

In [ ]:
!pip install ste